# This project aims to write an algorithm that can solve a sudoku board in a minimal amount of time

### This was accomplished by using two algorithms: backtrack search and AC3 arc consistency
Some boards can be solved using only the BTS, but others need AC3 combined with BTS to solve it

We will start by import some dependencies.

In [1]:
import numpy as np
import math
import time

#This function converts the given string input to a 9x9 numpy matrix
def readGridInput(gridStr):
    gridChar = np.zeros((9,9))
    gridChar = gridChar.astype(str)
    maxDomain = "123456789"
    for i in range(0,9):
        for j in range(0,9):
            gridChar[i][j] = gridStr[i*9+j]
            if gridChar[i][j] == "0":
                gridChar[i][j] = maxDomain
    return gridChar

The AC3 and BTS combination works by keeping track of all the possible values for each square given the constraints. Then it assumes the value if one of the squares by its value and removes all of the configurations that violate the constraints. If one of the squares has no remaining possible values, then the inital assumption is wrong and another is tried. AC3 is the one that removes configurations taht violate the constraints. BTS is like a DFS where it assumes the value of an element. It chooses what element to set using the Minimum Remaining values heuristic (MRV). It chooses the variables with least possible choices remaining.

In [2]:
#Replaces the grid of configurations by a 9x9 grid that represents the number of possible configurations in any square
def complexity(grid):
    complexityArr = np.zeros((9,9))
    for i in range(0,9):
        for j in range(0,9):
            complexityArr[i][j] = len(grid[i][j])
    return complexityArr

#Checks is the grid is solved. This is true is the number of possible configurations in all squares is 1
def isSolved(grid):
    complexityArr = complexity(grid)
    solved = True
    for i in range(0,9):
        for j in range(0,9):
            if not(complexityArr[i][j] == 1):
                solved = False
    return solved

#Checks a possible configuration if it satisfies the constraints of the board
def isValid(grid,i,j,newVal):
    grid[i][j] = newVal
    strColumn = ""
    strRow = ""
    strBlock = ""
    iBlock = math.floor(i/3)
    jBlock = math.floor(j/3)
    block = grid[3*iBlock:3*iBlock+3]
    block = block.transpose()[3*jBlock:3*jBlock+3]
    block = block.transpose()
    for x in range(0,9):
        if len(grid[i][x]) == 1:
            strRow += grid[i][x]
        if len(grid[x][j]) == 1:
            strColumn += grid[x][j]
    for x in range(0,3):
        for z in range(0,3):
            if len(block[x][z]) == 1:
                strBlock += block[x][z]
    if len(set(strRow)) != len(strRow):
        return False
    if len(set(strColumn)) != len(strColumn):
        return False
    if len(set(strBlock)) != len(strBlock):
        return False
    return True

#The main AC3 algorithm
def AC3(grid):
    changed = True
    while(changed):
        changed = False
        for i in range(0,9):
            for j in range(0,9):
                if len(grid[i][j])>1:
                    domain = list(grid[i][j])
                    newDomain = ""
                    for x in range(0,len(domain)):
                        if isValid(grid,i,j,domain[x]):
                            newDomain += domain[x]
                    if len(newDomain) == 0:
                        return False,-1
                    grid[i][j] = newDomain
                    if len(newDomain)<len(domain):
                        changed = True
    return isSolved(grid),grid

# A node class used in the BTS algorithm
class node:
    childs = None
    parent = None
    gridMap = None
    depth = None
    def __init__(self,childs,parent,gridMap,depth):
        self.childs = childs
        self.parent = parent
        self.gridMap = gridMap
        self.depth = depth

# Minimum Remaining values heuristic used to select which element to set next in the BTS
def MRV(grid):
    complexityArr = complexity(grid)
    minComplexity = 9
    iMinComplexity = -1
    jMinComplexity = -1
    for i in range(0,9):
        for j in range(0,9):
            if complexityArr[i][j] < minComplexity and complexityArr[i][j] > 1:
                minComplexity = complexityArr[i][j]
                iMinComplexity = i
                jMinComplexity = j
    return iMinComplexity,jMinComplexity

#BTS algorithm
def backtrack(parentNode):
    grid = parentNode.gridMap
    iToBranch, jToBranch = MRV(grid)
    domain = list(grid[iToBranch][jToBranch])
    children = []
    for i in range(0,len(domain)):
        newGrid = grid.copy()
        newGrid[iToBranch][jToBranch] = domain[i]
        done, answerGrid = AC3(newGrid)
        if done:
            return True,answerGrid
        elif type(answerGrid) != int:
            newNode = node(None,parentNode,answerGrid,parentNode.depth+1)
            children.append(newNode)
    parentNode.children = children
    for i in range(0,len(children)):
        doneBK, answerGridBK = backtrack(children[i])
        if doneBK:
            return True, answerGridBK
    return False, -1

### Main solver function:

If you want to try the algorithm on a board, arg1 is the unsolved board. This is converted into one string by reading each row of the board and combining all of the rows into one string. A blank space is represented with a 0.

In [3]:
# Runs AC3 and BTS. returns the correct answer and if only AC3 was used it returns AC3,
# if BTS was also used with AC3 it returns BTS. It also returns the time used to execute the full algorithm

def solveGrid(grid):
    startTime = time.time()
    is_solved, answer = AC3(grid)
    finalAnswer = answer
    startNode = node(None,None,answer,0)
    if not is_solved:
        is_solvedBK, answerBK = backtrack(startNode)
        finalAnswer = answerBK

    finalAnswerStr = ""
    for i in range(0,9):
        for j in range(0,9):
            finalAnswerStr += finalAnswer[i][j]
    endTime = time.time()
    if is_solved:
        return finalAnswerStr+" AC3",finalAnswer,endTime-startTime
    return finalAnswerStr+" BTS",finalAnswer,endTime-startTime


arg1 = "005300000800000020070010500400005300010070006003200080060500009004000030000009700"

UnsolvedGrid = readGridInput(arg1)
answerStr, answerGrid, timeToRun = solveGrid(UnsolvedGrid)

print(answerStr)
print(answerGrid)

print("Time to run: "+ str(timeToRun)+" seconds")

145327698839654127672918543496185372218473956753296481367542819984761235521839764 BTS
[['1' '4' '5' '3' '2' '7' '6' '9' '8']
 ['8' '3' '9' '6' '5' '4' '1' '2' '7']
 ['6' '7' '2' '9' '1' '8' '5' '4' '3']
 ['4' '9' '6' '1' '8' '5' '3' '7' '2']
 ['2' '1' '8' '4' '7' '3' '9' '5' '6']
 ['7' '5' '3' '2' '9' '6' '4' '8' '1']
 ['3' '6' '7' '5' '4' '2' '8' '1' '9']
 ['9' '8' '4' '7' '6' '1' '2' '3' '5']
 ['5' '2' '1' '8' '3' '9' '7' '6' '4']]
Time to run: 0.43335747718811035 seconds
